In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

# Define the data
data = {
    "Sentence": [
        "The product exceeded my expectations.",
        "The service was terrible and frustrating.",
        "I'm not sure how I feel about the new update.",
        "This is the best meal I've ever had!",
        "The experience was just okay, nothing special.",
        "I love the way this app works, so intuitive!",
        "The movie was boring and too long.",
        "The customer support was helpful and responsive.",
        "I don't really have an opinion on this matter.",
        "The package arrived late and damaged.",
        "Everything went smoothly, no issues at all.",
        "The food was mediocre, not worth the price.",
        "I'm satisfied with the purchase.",
        "The meeting was unproductive and a waste of time.",
        "It's an average product, nothing to rave about.",
        "The weather was perfect for our trip.",
        "The staff was rude and unprofessional.",
        "I don't have strong feelings either way.",
        "The new policy is fair and well thought out.",
        "The software keeps crashing and it's very annoying.",
        "I'm indifferent to the changes they've made.",
        "The vacation was a dream come true!",
        "The instructions were unclear and confusing.",
        "I'm okay with the final decision.",
        "The performance was outstanding!",
        "The website is difficult to navigate.",
        "I'm neither happy nor upset about the results.",
        "The event was well-organized and enjoyable.",
        "The product didn't work as advertised.",
        "The service was satisfactory, nothing to complain about."
    ],
    "Sentiment": [
        2, 0, 1, 2, 1, 2, 0, 2, 1, 0, 2, 0, 2, 0, 1, 2, 0, 1, 2, 0,
        1, 2, 0, 1, 2, 0, 1, 2, 0, 1
    ]
}

# Create the DataFrame
df = pd.DataFrame(data)
df1 = df.copy()

# Display the DataFrame
df.head()

,Sentence,Sentiment
0,The product exceeded my expectations.,2
1,The service was terrible and frustrating.,0
2,I'm not sure how I feel about the new update.,1
3,This is the best meal I've ever had!,2
4,"The experience was just okay, nothing special.",1


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import pandas as pd
import numpy as np
from scipy.special import softmax

## cardiffnlp/twitter-roberta-base-sentiment-latest

In [4]:
# Load the model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Preprocess text
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [6]:
# Initialize lists to store the sentiment predictions and scores
predictions = []
negative_scores = []
neutral_scores = []
positive_scores = []

# Loop through each sentence, preprocess it, and predict sentiment
for sentence in df['Sentence']:
    # Preprocess the sentence
    text = preprocess(sentence)
    
    # Tokenize and encode the sentence
    encoded_input = tokenizer(text, return_tensors='pt')
    
    # Get model predictions
    output = model(**encoded_input)
    
    # Compute softmax to get the probabilities
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    # Get the label with the highest score
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    label = config.id2label[ranking[0]]
    
    # Append the label and scores to the respective lists
    predictions.append(label)
    negative_scores.append(scores[0])
    neutral_scores.append(scores[1])
    positive_scores.append(scores[2])

# Add the predicted sentiments and scores to the DataFrame
df['Predicted_Sentiment'] = predictions
df['Negative_Score'] = negative_scores
df['Neutral_Score'] = neutral_scores
df['Positive_Score'] = positive_scores

In [7]:
# Display the updated DataFrame
df.head(10)

,Sentence,Sentiment,Predicted_Sentiment,Negative_Score,Neutral_Score,Positive_Score
0,The product exceeded my expectations.,2,positive,0.003518,0.020787,0.975695
1,The service was terrible and frustrating.,0,negative,0.940156,0.052094,0.007750
2,I'm not sure how I feel about the new update.,1,negative,0.794810,0.194431,0.010759
3,This is the best meal I've ever had!,2,positive,0.006072,0.008270,0.985658
4,"The experience was just okay, nothing special.",1,positive,0.070396,0.393959,0.535646
5,"I love the way this app works, so intuitive!",2,positive,0.004777,0.008495,0.986728
6,The movie was boring and too long.,0,negative,0.907798,0.083041,0.009161
7,The customer support was helpful and responsive.,2,positive,0.006941,0.034304,0.958756
8,I don't really have an opinion on this matter.,1,negative,0.555244,0.431034,0.013721
9,The package arrived late and damaged.,0,negative,0.871360,0.120649,0.007991


In [8]:
# Map predicted sentiments to numerical values
sentiment_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
df['Predicted_Sentiment_Numeric'] = df['Predicted_Sentiment'].map(sentiment_mapping)

# Calculate accuracy
correct_predictions = (df['Sentiment'] == df['Predicted_Sentiment_Numeric']).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions

print(f"Accuracy: {accuracy:.2f} or {accuracy * 100:.2f}%")

Accuracy: 0.70 or 70.00%


## finiteautomata/bertweet-base-sentiment-analysis

In [9]:
# Load the model and tokenizer
MODEL = "finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [10]:
# Initialize lists to store predictions and scores
predictions = []
pos_scores = []
neg_scores = []
neu_scores = []

# Loop through each sentence, preprocess, and predict sentiment
for sentence in df1['Sentence']:
    # Tokenize and encode the sentence
    encoded_input = tokenizer(sentence, return_tensors='pt')
    
    # Get model predictions
    output = model(**encoded_input)
    
    # Compute softmax to get the probabilities
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    # Get the label with the highest score
    label_id = scores.argmax()
    label = model.config.id2label[label_id]
    
    # Append the label and scores to the respective lists
    predictions.append(label)
    neg_scores.append(scores[0])
    neu_scores.append(scores[1])
    pos_scores.append(scores[2])

# Add the predicted sentiments and scores to the DataFrame
df1['Predicted_Sentiment'] = predictions
df1['Negative_Score'] = neg_scores
df1['Neutral_Score'] = neu_scores
df1['Positive_Score'] = pos_scores

In [11]:
# Display the updated DataFrame
df1.head(10)

,Sentence,Sentiment,Predicted_Sentiment,Negative_Score,Neutral_Score,Positive_Score
0,The product exceeded my expectations.,2,NEG,0.644661,0.211589,0.143750
1,The service was terrible and frustrating.,0,NEG,0.983342,0.012225,0.004433
2,I'm not sure how I feel about the new update.,1,NEG,0.600196,0.391913,0.007892
3,This is the best meal I've ever had!,2,POS,0.003815,0.004605,0.991580
4,"The experience was just okay, nothing special.",1,NEG,0.968708,0.027287,0.004004
5,"I love the way this app works, so intuitive!",2,POS,0.002397,0.004691,0.992912
6,The movie was boring and too long.,0,NEG,0.978426,0.017900,0.003674
7,The customer support was helpful and responsive.,2,POS,0.001523,0.010864,0.987613
8,I don't really have an opinion on this matter.,1,NEU,0.334240,0.658680,0.007080
9,The package arrived late and damaged.,0,NEG,0.946913,0.050136,0.002951


In [12]:
# Map predicted sentiments to numerical values
sentiment_mapping = {'NEG': 0, 'NEU': 1, 'POS': 2}
df1['Predicted_Sentiment_Numeric'] = df1['Predicted_Sentiment'].map(sentiment_mapping)

# Calculate accuracy
correct_predictions = (df1['Sentiment'] == df1['Predicted_Sentiment_Numeric']).sum()
total_predictions = len(df1)
accuracy = correct_predictions / total_predictions

print(f"Accuracy: {accuracy:.2f} or {accuracy * 100:.2f}%")

Accuracy: 0.70 or 70.00%
